In [1]:
!pip install weave openai

INFO: pip is looking at multiple versions of gql to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.6/375.6 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 553.2/553.2 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.7/310.7 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 67.0 MB/s 

In [2]:
import weave

weave.init("rag-baselining")

Please login to Weights & Biases (https://wandb.ai/) to continue:


wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Logged in as Weights & Biases user: sohom377.
View Weave data at https://wandb.ai/sohom377-zemuria/rag-baselining/weave


In [4]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

In [6]:
from openai import OpenAI

client = OpenAI()
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "Please translate English into Hindi."},
        {"role": "user", "content": "Football is the best game."},
    ],
    temperature=0,
)


generation = response.choices[0].message.content
print(generation)

🍩 https://wandb.ai/sohom377-zemuria/rag-baselining/r/call/0192279e-948c-7a31-b800-e4aa62582b99
फुटबॉल सबसे अच्छा खेल है।


In [8]:
@weave.op()
def translation(user_input):
    client = OpenAI()
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "Please translate English into Hindi."},
            {"role": "user", "content": user_input},
        ],
        temperature=0,
    )
    return response.choices[0].message.content

result = translation("Football is the best game")
print(result)

🍩 https://wandb.ai/sohom377-zemuria/rag-baselining/r/call/0192279f-d68d-7dd1-b21c-3ce241852b94
फुटबॉल सबसे अच्छा खेल है।


In [9]:
from openai import OpenAI

class TranslationModel(weave.Model):
    system_instruction: str

    @weave.op()
    def translation(self, user_input):
        client = OpenAI()
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": self.system_instruction},
                {"role": "user", "content": user_input},
            ],
            temperature=0,
        )
        return response.choices[0].message.content


model = TranslationModel(
    system_instruction="Please translate English into Hindi.",
)
result = model.translation("Generative AI has been hyped up a lot.")
print(result)

🍩 https://wandb.ai/sohom377-zemuria/rag-baselining/r/call/019227a0-03c6-7301-9d31-1215e0988f10
जनरेटिव एआई को बहुत अधिक प्रचारित किया गया है।


In [10]:
documents = [
    "Manchester United is a professional football club based in Manchester, England. It is one of the most successful clubs in football history, known for its multiple Premier League titles and UEFA Champions League victories. The club has a global fanbase and is celebrated for its iconic players and achievements.",
    "Real Madrid is a Spanish football club renowned for its dominance in European football. It holds the record for the most UEFA Champions League titles and has played a significant role in shaping the landscape of international football. The club is a symbol of excellence in La Liga.",
    "FC Barcelona is a Spanish football club famous for its attractive style of play, called 'tiki-taka,' and its intense rivalry with Real Madrid. The club has won numerous La Liga and UEFA Champions League titles, contributing to the legacy of some of the greatest football players in history.",
    "Liverpool FC is a historic football club based in Liverpool, England. It has won multiple Premier League and UEFA Champions League titles, known for its passionate supporters and strong footballing culture. The club is one of the most successful in both English and European football.",
    "Bayern Munich is a professional football club based in Munich, Germany. It is the most successful team in Bundesliga history, boasting numerous domestic titles as well as multiple UEFA Champions League victories. The club is known for its consistency and dominance in European competitions.",
    "Juventus is a professional football club based in Turin, Italy. It is the most successful club in Italian football history, dominating Serie A and regularly competing in European competitions, including the UEFA Champions League."
]


In [11]:
from openai import OpenAI


def docs_to_embeddings(docs: list) -> list:
    openai = OpenAI()
    document_embeddings = []
    for doc in docs:
        response = (
            openai.embeddings.create(
                input=doc,
                model="text-embedding-3-small"
            )
            .data[0]
            .embedding
        )
        document_embeddings.append(response)
    return document_embeddings


docs_embeddings = docs_to_embeddings(documents)

In [12]:
import numpy as np


@weave.op()
def get_most_relevant_document(query):

    openai = OpenAI()
    query_embedding = (
        openai.embeddings.create(
            input=query,
            model="text-embedding-3-small"
        )
        .data[0]
        .embedding
    )


    similarities = [
        np.dot(query_embedding, doc_emb)
        / (np.linalg.norm(query_embedding) * np.linalg.norm(doc_emb))
        for doc_emb in docs_embeddings
    ]


    most_relevant_doc_index = np.argmax(similarities)
    return documents[most_relevant_doc_index]

In [13]:
class RAGModel(weave.Model):
    model_name: str = "gpt-4o"


    @weave.op()
    def predict(self, question: str) -> dict:

        context = get_most_relevant_document(question)


        client = OpenAI()
        query = f"""Please answer the questions using only the following context.
If you don't know the answer, answer 'I don't know.'


context:"
```
{context}
```

question:
{question}"""
        response = client.chat.completions.create(
            model=self.model_name,
            messages=[
                {"role": "user", "content": query},
            ],
            temperature=0.0,
            response_format={"type": "text"},
        )
        answer = response.choices[0].message.content
        return {"answer": answer, "context": context}

In [14]:
model = RAGModel(
    model_name="gpt-4o"
)
model.predict("What is Manchester United?")

🍩 https://wandb.ai/sohom377-zemuria/rag-baselining/r/call/019227a2-9748-7113-bb3f-aefc2ba217c8


{'answer': 'Manchester United is a professional football club based in Manchester, England.',
 'context': 'Manchester United is a professional football club based in Manchester, England. It is one of the most successful clubs in football history, known for its multiple Premier League titles and UEFA Champions League victories. The club has a global fanbase and is celebrated for its iconic players and achievements.'}

In [15]:
questions = [
    {"question": "Which football club is based in Manchester, England?"},
    {"question": "Which football club holds the record for the most UEFA Champions League titles?"},
    {"question": "Which football club is known for the tiki-taka style of play and its rivalry with Real Madrid?"},
    {"question": "Which English football club has passionate supporters and a history of success in both the Premier League and UEFA Champions League?"},
    {"question": "Which football club is the most successful in Bundesliga history?"},
    {"question": "Which Italian football club dominates Serie A and regularly competes in the UEFA Champions League?"}
]

In [16]:
import json

@weave.op()
async def context_precision_score(question, model_output):
    context_precision_prompt = """Given the question, answer, and context, verify whether the context helped in reaching the provided answer.
In the JSON output, if the context was useful, output {{verification: 1}}.
If the context was not useful, output {{verification: 0}}.
Output in valid JSON format only.

Context:
```
{context}
```

Answer:
{answer}
"""

    client = OpenAI()
    prompt = context_precision_prompt.format(
        question=question,
        context=model_output["context"],
        answer=model_output["answer"],
    )
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}],
        response_format={ "type": "json_object" }
    )
    response_message = response.choices[0].message
    response = json.loads(response_message.content)
    return {
        "verdict": int(response["verification"]) == 1,
    }

In [17]:
import nest_asyncio
nest_asyncio.apply()

In [18]:
evaluation = weave.Evaluation(
    dataset=questions,
    scorers=[context_precision_score]
)

In [19]:
model = RAGModel(
    model_name="gpt-4o"
)
await evaluation.evaluate(model)

Evaluated 1 of 6 examples

Evaluated 2 of 6 examples

Evaluated 3 of 6 examples

Evaluated 4 of 6 examples

Evaluated 5 of 6 examples

Evaluated 6 of 6 examples

Evaluation summary
{
    'context_precision_score': {'verdict': {'true_count': 5, 'true_fraction': 0.8333333333333334}},
    'model_latency': {'mean': 2.678079605102539}
}

🍩 https://wandb.ai/sohom377-zemuria/rag-baselining/r/call/019227a3-efe1-76b0-a1a0-faea5084a533


{'context_precision_score': {'verdict': {'true_count': 5,
   'true_fraction': 0.8333333333333334}},
 'model_latency': {'mean': 2.678079605102539}}

In [20]:
model = RAGModel(
    model_name="gpt-4o-mini"
)
await evaluation.evaluate(model)

Evaluated 1 of 6 examples

Evaluated 2 of 6 examples

Evaluated 3 of 6 examples

Evaluated 4 of 6 examples

Evaluated 5 of 6 examples

Evaluated 6 of 6 examples

Evaluation summary
{
    'context_precision_score': {'verdict': {'true_count': 6, 'true_fraction': 1.0}},
    'model_latency': {'mean': 2.7649885416030884}
}

🍩 https://wandb.ai/sohom377-zemuria/rag-baselining/r/call/019227a4-193e-7f22-bd08-3ae7221171c8


{'context_precision_score': {'verdict': {'true_count': 6,
   'true_fraction': 1.0}},
 'model_latency': {'mean': 2.7649885416030884}}